In [11]:
import a1chemy.data_source as data_source
import a1chemy.indicators as indicators
import a1chemy.plot.plotly_tools as plotly_tools
import pymongo
import operator
import pandas as pd
from tqdm.notebook import tqdm
import base64

mongo_client = pymongo.MongoClient("mongodb://localhost:27017/", username='a1chemy', password='1B2C9046-E3CC-447F-9961-E125759BA44F')
mongo_ticks_client = data_source.MongoTicks(mongo_client)
mongo_tags_client = data_source.MongoTags(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)
mongo_fund_client = data_source.MongoFund(mongo_client)

In [12]:
exchange='SH'
symbol='SH510300'
ticks = mongo_ticks_client.find_one(exchange=exchange, symbol=symbol)
fund = mongo_fund_client.find_one(exchange=exchange, symbol=symbol)

In [37]:
import ipywidgets as widgets

count=20

thumbnail = plotly_tools.ticks_thumbnail(ticks=ticks, start=-count)
shares = plotly_tools.fund_amount_thumbnail(fund=fund, start=-count)

layouts = [
    widgets.Layout(width='80px'),
    widgets.Layout(width='90px', display='flex', justify_content="center"),
    widgets.Layout(width='40px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center"),
    widgets.Layout(width='120px', display='flex', justify_content="center")
]

title=widgets.HBox([widgets.Label(value='Broad Market', layout=layouts[0]), 
                    widgets.Label(value='SYMBOL', layout=layouts[1]),
                    widgets.Label(value='5D%', layout=layouts[2]), 
                    widgets.Label(value='1M', layout=layouts[3]), 
                    widgets.Label(value='shares', layout=layouts[4])])
value1=widgets.HBox([widgets.Label(value=ticks.name, layout=layouts[0]), 
                     widgets.Label(value=ticks.symbol, layout=layouts[1]), 
                     widgets.Label(value='5', layout=layouts[2]), 
                     widgets.Image(value=thumbnail.data, layout=layouts[3]),
                     widgets.Image(value=shares.data, layout=layouts[4])])

widgets.VBox([title, value1])